## decoding demo

In [1]:
from collections import Counter
from preprocessing import CharacterIndexer, SlotIndexer, IntentIndexer
from gensim.models import Word2Vec
import json
import numpy as np
import pandas as pd
import pickle
import h5py
import math

/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


we can load the layers from the `json` file so we only need the loading functions and the custom layers

In [2]:
from keras_contrib.layers import CRF
from keras_contrib.utils import save_load_utils
from kutilities.layers import AttentionWithContext
from keras.models import model_from_json

Using TensorFlow backend.
/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
sentindexer = pickle.load(open('encoded/atis_sentindexer.pkl', 'rb'))
slotindexer = pickle.load(open('encoded/atis_slotindexer.pkl', 'rb'))
intindexer  = pickle.load(open('encoded/atis_intindexer.pkl',  'rb'))

## model loading

due to the `keras-contrib` CRF and added attention layers, the easiest way to load the model is to recreate it and load the weights.

In [4]:
modelname = 'test_model'

In [5]:
import keras
with open('model/'+modelname+'.json') as f:
    json_string = f.read()
    
model = model_from_json(json_string, custom_objects={"CRF": CRF, "AttentionWithContext" : AttentionWithContext})

model.load_weights('model/'+modelname+'.h5')

### test loaded model

In [6]:
tst_text_idx = np.load('encoded/tst_text_idx.npy')
tst_char_idx = np.load('encoded/tst_char_idx.npy')
tst_slot_idx = np.load('encoded/tst_slot_idx.npy')
tst_int_idx  = np.load('encoded/tst_int_idx.npy')

In [7]:
prd_slots_dist, prd_ints_dist = model.predict([tst_text_idx, tst_char_idx])
prd_int_idx  = np.squeeze(np.argmax(prd_ints_dist, axis=-1))
prd_slot_idx = np.argmax(prd_slots_dist, axis=-1)
prd_ints = intindexer.inverse_transform(prd_int_idx)
tru_ints = intindexer.inverse_transform(tst_int_idx)

In [8]:
from sklearn.metrics import accuracy_score
print('INTENT ACC:  ', accuracy_score(tru_ints, prd_ints))

INTENT ACC:   0.9630459126539753


/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/derek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### decoding functions

format the input string properly (lower-case, add BOS and EOS tags, strip punctuation, and index), predict on the model, use `argmax` to get predictions then `inverse_transform()` back into human-readable labels

a more convenient way to do this would be to encapsulate all the above and below into a class.

In [9]:
import re
def preprocess(snt):
    snt = snt.lower()
    snt = re.sub(r'[^0-9a-z\s]', '', snt)
    snt = snt.split()
    snt = ['BOS'] + snt + ['EOS']
    snt = [snt]
    out = sentindexer.transform(snt)
    return snt, out[0], out[1]

In [10]:
def predict(s):
    tk, wt, ct = preprocess(s)
    tk = tk[0]
    sp, ip = model.predict([wt, ct])
    sp = np.argmax(sp, axis=-1)
    ip = np.argmax(ip, axis=-1)
    sp = slotindexer.inverse_transform(np.expand_dims(sp, axis=-1))[0]
    sp = [x.split('-')[-1] for x in sp]
    
    spd = {}
    for i, p in enumerate(sp):
        if p != 'O':
            if p in spd.keys():
                spd[p].append(tk[i])
            else:
                spd[p] = []
                spd[p].append(tk[i])
    
    spo = {}
    for k in spd.keys():
        spo[k] = ' '.join(spd[k])
    
    ip = intindexer.inverse_transform([ip]+[[0]])[0]

    print('query:', s)
    print('slots:')
    print(spo)
    print('intent:', ip)
    
    return spo, ip
    

### test

In [11]:
inpt = "looking for direct flights from Chicago to LAX"
a, b = predict(inpt)

query: looking for direct flights from Chicago to LAX
slots:
{'connect': 'direct', 'fromloc.city_name': 'chicago', 'toloc.city_name': 'lax'}
intent: atis_flight


In [12]:
inpt = "give me flights and fares from New York to Dallas"
a, b = predict(inpt)

query: give me flights and fares from New York to Dallas
slots:
{'fromloc.city_name': 'new york', 'toloc.city_name': 'dallas'}
intent: atis_flight#atis_airfare


In [13]:
inpt = "i want a first class flight to los angeles"
a, b = predict(inpt)

query: i want a first class flight to los angeles
slots:
{'class_type': 'first class', 'toloc.city_name': 'los angeles'}
intent: atis_flight
